# TOC

__Chapter 7 - Generative Networks__

1. [Import](#Import)
1. [Neural style transfer](#Neural-style-transfer)
    1. [Loading the data](#Loading-the-data)
    1. [Creating the VGG model](#Creating-the-VGG-model)
    1. [Content loss](#Content-loss)
    1. [Style loss](#Style-loss)
    1. [Extracting the losses](#Extracting-the-losses)
    1. [Creating loss function for each layers](#Creating-loss-function-for-each-layers)
    1. [Creating the optimizer](#Creating-the-optimizer)
    1. [Training](#Training)
1. [Generative adversarial networks](#Generative-adversarial-networks)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)
1. [](#)

# Import

<a id = 'Import'></a>

In [ ]:
# Standard libary and settings
import os
import sys
import importlib
import itertools
import warnings; warnings.simplefilter('ignore')
from IPython.core.display import display, HTML; display(HTML("<style>.container { width:95% !important; }</style>"))

# Data extensions and settings
import numpy as np
np.set_printoptions(threshold = np.inf, suppress = True)
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.options.display.float_format = '{:,.6f}'.format

# pytorch tools
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
from torch.autograd import Variable
from torchvision import datasets, models, transforms

# Visualization extensions and settings
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
sns.set_style('whitegrid')


# Neural style transfer

Given a content image and a style image, generate a new image that combines the content of the content image and the style of the style image.

The style of an image is captured across multiple layers in a CNN by a technique called gram matrix. This calculates the correlation between the features maps captures across each layer. Similarly styled images have similar values for a gram matrix.



<a id = 'Neural-style-transfer'></a>

## Loading the data



<a id = 'Loading-the-data'></a>

In [ ]:
# fix the image size
imsize = 512
is_cuda = torch.cuda.is_available()

# convert image for training with VGG model
prep = transforms.Compose([transforms.Resize(imsize)
                          ,transforms.ToTensor()
                          ,transforms.Lambda(lambda x: x[torch.LongTensor([2, 1, 0])]) # turn to BGR
                          ,transforms.Normalize(mean = [0.40760392, 0.45795686, 0.48501961]
                                               std = [1, 1, 1]) # subtract imagenet mean 
                          ,transforms.Lambda(lambda x: x.mul_(255))
                          ])

# convert the generated image back to a format that can be visualized
postpa = transforms.Compose([transforms.Lambda(lambda x: x.mul_(1. / 255))
                            ,transforms.Normalize(mean = [0.40760392, 0.45795686, 0.48501961]
                                                 ,std = [1, 1, 1])
                            ,transforms.Lambda(lambda x: x[torch.LongTensor([2, 1, 0 ])]) # turn to RGB
                            ])
postpb = transforms.Compose([transform.ToPILImage()])

# ensure data in the image does not cross the permissible range of values
def postp(tensor):
    t = postpa(tensor)
    t[t > 1] = 1
    t[t < 0 ] = 0
    img = postpb(t)
    return img

# ease data loading
def image_loader(image_name):
    image = Image.open(image_name)
    image = Variable(prep(image))
    image = image.unsqueeze(0)
    return image



In [ ]:
#
style_img = image_loader('images/vangogh_starry_night.jpg')
convert_img = image_loader('images/tuebinge_neckarfront.jpg')

opt_img = Variable(content_img.data.clone(), requires_grad = True)


## Creating the VGG model



<a id = 'Creating-the-VGG-model'></a>

In [ ]:
# create a VGG model, grabbing only the convolution block (features) and freeze the parameters
vgg = vgg19(pretrained = True).features

for param in vgg.parameters():
    param.requires_grad = False


## Content loss


<a id = 'Content-loss'></a>

In [ ]:
# 
target_layer = dummy_fn(content_img)
noise_layer = dummy_fn(noise_img)
criterion = nn.MSELoss()
content_loss = criterion(target_layer, noise_layer)


## Style loss

Style loss is the MSE of the gram matrix generated for each feature map. Envision a feature map with dimensions representing bacth_size by color channels and values (which in this example is itself a 3 by 3 window). To calculate the gram matrix, the 9 values in each channel are flattened into a 9 value vector, and then the correlation coefficient is calculated by multiplying the flattened vector by its transpose.

The class below is written in a way so that it can be used like another PyTorch layer.  First, the batch, channel, height and width are maintained, and then the features are reshaped such that the batch and channel dimensions remain intact, and the values are flattened along the height and width dimension. The gram matrix is calculated using the PyTorch batch matrix multiplication function torch.bmm(), which will multiply the flattened values with its transposed vector. The final step normalizes the values of the gram matrix by dividing it by the number of elements. Without this, a feature map with an especially high number of values would tend to dominate the score.

The second class below calculates the style loss, which is also implemented as a PyTorch layer. It calculates the MSE between the input image gram matrix and the style image gram matrix

<a id = 'Style-loss'></a>

In [ ]:
# 
class GramMatrix(nn.Module):
    def forward(self, input):
        b, c, h, w = input.size()
        features = input.view(b, c, h * w)
        gram_matrix = torch.bmm(features, features.tranpose(1, 2))
        gram_matrix.div_(h * w)
        return gram_matrix

class StyleLoss(nn.Module):
    def forward(self, inputs, targets):
        out = nn.MSELoss()(GramMatrix()(inputs), targets)
        return (out)
    

## Extracting the losses

Just as activations can be extracted from convolution layers use the register_forward_hook(), we can extract losses of different convolutional layers required to calculate style loss and content loss. The key difference is that rather than extracting from a single layer, we need to extract outputs from several layers.

In the class below, the init method takes in the model on which we will call register_forward_hook() as well as the layer ID number for layers from which we will extract the outputs. The init method's for loop iterates through the layer IDs and registers the forward hook required the pull outputs.

hook_fn is passed to register_forward_hook() and is called by PyTorch after the current layer is registered. Inside the function, the output is captured and stored in the features array.

Lastly, the remove function is called to clear the outputs captured, otherwise this process may result in memory issues.

The extract_layers function extracts the outputs for the style and content images. Inside this funciton, we call LayerActivations and pass in the model and the layer numbers. We follow this by ensuring we have an empty list. The an image is passed through the model, and we will review the outputs generated in the features array 


<a id = 'Extracting-the-losses'></a>

In [1]:
# 
class LayerActivations():
    features = []
    
    def __init__(self, model, layer_nums):
        self.hooks = []
        for layer_num in layer_nums:
            self.hooks.append(model[layer_num].register_forward_hook(self.hook_fn))
            
    def hook_fn(self, module, input, output):
        self.features.append(output)
        
    def remove(self):
        for hook in self.hooks:
            hook.remove()

def extract_layers(layers, img, model = None):
    la = LayerActivations(model, layers)
    la.features = []
    out = model(img)
    la.remove()
    return la.features


In [ ]:
# the outputs needs to be detached from the graphs that created them
content_targets = extract_layers(content_layers, content_img, model = vgg)
style_targets = extract_layers(style_layers, style_img, model = vgg)

content_targets [t.detach() for t in content_targets]
style_targets = [GramMatrix()(t).detach() for t in style_targets]

# add all targets into one list
target = style_targets + content_targets

# specify layers to be extracted
style_layers = [1, 6, 11, 20, 25]
content_layers = [21]
loss_layers = style_layers + content_layers


In [ ]:
# the optimizer needs a single scalar to minimize, so the losses frome each layer are summed
style_weights = [1e3 / n**2 for n in [64, 128, 256, 512, 512]]
content_weights = [1e0]
weights = style_weights + content_weights


In [ ]:
# review layers selected
print(vgg)


## Creating loss function for each layers

We need to create the loss layers for the separate style losses and content losses. The variable loss_fns is a list containing several style loss objects and content loss objects that are based on the lengths of the arrays created.



<a id = 'Creating-loss-function-for-each-layers'></a>

In [ ]:
# 
loss_fns = [StyleLoss()] * len(style_layers) + [nn.MSELoss()] * len(content_layers)


## Creating the optimizer

An optimizer typically receives the parameters of the model, but in this case we are using VGG models as feature extracts, and therefore cannot pass the VGG parameters. Instead, we will provide the parameters of the opt_img variable. These are the parameters that will be optimized to make the image have the required content and style.



<a id = 'Creating-the-optimizer'></a>

In [ ]:
# 
optimizer = optim.LBFGS([opt_img])


## Training

This training method will calculate loss for multiple layers. Each time the optimizer is called, it will chang the input image so that the content and style gets nearer to the target's content and style.

In the function below, each iteration involves calcualting the output from different layers of the VGG model using extract_layers. The only values that change here are the values of the style image (opt_img). Once the outputs are calculated, we calcualte the losses by iterations through the outputs and passing them to the associated loss functions along with the targets. The losses are summed and the backward function is called.


<a id = 'Training'></a>

In [ ]:
# 
max_iter = 500
show_iter = 50
n_iter = [0]

while n_iter[0] <= max_iter:
    def closure()
        optimizer.zero_grad()
        
        out = extract_layers(loss, opt_img, model = vgg)
        layer_losses = [weights[a] * loss_fns[a](A, targets[a]) for a, A in enumerate(out)]
        loss = sum(layer_losses)
        loss.backward()
        n_iter[0] += 1
        
        print(loss)
        
        if n_iter[0] % show_iter == (show_iter - 1):
            print('Iteration: {}, loss: {}'.format(n_iter[0] + 1, loss.data[0]))
    
    return loss

optimizer.step(closure)


# Generative adversarial networks



<a id = 'Generative-adversarial-networks'></a>

In [ ]:
# 



# A



<a id = ''></a>

In [ ]:
# 



# A



<a id = ''></a>

In [ ]:
# 



# A



<a id = ''></a>

In [ ]:
# 



# A



<a id = ''></a>

In [ ]:
# 



# A



<a id = ''></a>

In [ ]:
# 



# A



<a id = ''></a>

In [ ]:
# 

